<a href="https://colab.research.google.com/github/OmarShafie/MNIST-HBKU/blob/master/bert_tensorflow_2_huggingface_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toxic Comment Classification Challenge
## BERT - TensorFlow 2 & Hugging Face Transformers Library

In [ ]:
!nvidia-smi

Tue Dec  8 18:09:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install -q kaggle
from google.colab import files
# Upload the API Token
files.upload()
#make sure you have accepted the rules of the competetion 
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!ls -l

Saving kaggle.json to kaggle.json
 85% 20.0M/23.4M [00:00<00:00, 42.7MB/s]
100% 23.4M/23.4M [00:00<00:00, 52.9MB/s]
 95% 25.0M/26.3M [00:00<00:00, 42.8MB/s]
100% 26.3M/26.3M [00:00<00:00, 48.9MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 203MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 201MB/s]
total 53908
-rw-r--r-- 1 root root       66 Dec  8 18:09 kaggle.json
drwxr-xr-x 1 root root     4096 Dec  2 22:04 sample_data
-rw-r--r-- 1 root root  1459715 Dec  8 18:09 sample_submission.csv.zip
-rw-r--r-- 1 root root 24577258 Dec  8 18:09 test.csv.zip
-rw-r--r-- 1 root root  1527605 Dec  8 18:09 test_labels.csv.zip
-rw-r--r-- 1 root root 27619914 Dec  8 18:09 train.csv.zip


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 22.1MB/s 
     |████████████████████████████████| 890kB 53.0MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0625a4d535327f41028abf6f4ffab3b22031d4e8da9d66682708071ab5523f78
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
dataset_directory = '.'

In [ ]:
!mkdir data
!unzip {dataset_directory}/train.csv.zip -d data/
!unzip {dataset_directory}/test.csv.zip  -d data/
!unzip {dataset_directory}/test_labels.csv.zip  -d data/
!unzip {dataset_directory}/sample_submission.csv.zip  -d data/
!ls

Archive:  ./train.csv.zip
  inflating: data/train.csv          
Archive:  ./test.csv.zip
  inflating: data/test.csv           
Archive:  ./test_labels.csv.zip
  inflating: data/test_labels.csv    
Archive:  ./sample_submission.csv.zip
  inflating: data/sample_submission.csv  
data	     sample_data		test.csv.zip	     train.csv.zip
kaggle.json  sample_submission.csv.zip	test_labels.csv.zip


In [ ]:
!ls ./data

sample_submission.csv  test.csv  test_labels.csv  train.csv


In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

## 1. Data Pipeline
- Loading the datasets from CSVs
- Preprocessing (Tokenization, Truncation & Padding)
- Creating efficient data pipelines using tf.data

In [ ]:
train_path = 'data/train.csv'
test_path = 'data/test.csv'
test_labels_path = 'data/test_labels.csv'
subm_path = 'data/sample_submission.csv'

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_test_labels = pd.read_csv(test_labels_path)
df_test_labels = df_test_labels.set_index('id')

df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

In [ ]:
input_ids = tokenize_sentences(df_train['comment_text'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
from sklearn.model_selection import train_test_split

labels =  df_train[label_cols].values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_inputs)
validation_size = len(validation_inputs)

In [ ]:
BATCH_SIZE = 32
NR_EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

In [ ]:
train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

## 2. BERT Model
- Load the pretrained BERT base-model from Transformers library
- Take the first hidden-state from BERT output (corresponding to CLS token) and feed it into a Dense layer with 6 neurons and sigmoid activation (Classifier). The outputs of this layer can be interpreted as probabilities for each of the 6 classes.

In [ ]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

In [ ]:
model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## 3. Training Loop
- Use BinaryCrossentropy as loss function (is calculated for each of the output 6 output neurons ...that's like training 6 binary classification tasks at the same time) 
- Use the AdamW optimizer with 1-cycle-policy from the Transformers library
- AUC evaluation metrics

In [ ]:
import time
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
optimizer,_ = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    grads_vars = [(gradients[i], model.trainable_variables[i]) for i in range(len(gradients))]
    optimizer.apply_gradients(grads_vars)

    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
        
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
                                              
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')
train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

================================================== EPOCH 0 ==================================================



Train Step: 0, Loss: 0.8298330903053284
toxic roc_auc 0.19354841113090515
severe_toxic roc_auc 0.0
obscene roc_auc 0.0
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 1000, Loss: 0.17293472588062286
toxic roc_auc 0.8852025866508484
severe_toxic roc_auc 0.825751006603241
obscene roc_auc 0.9093009829521179
threat roc_auc 0.6697405576705933
insult roc_auc 0.8453302979469299
identity_hate roc_auc 0.7649480104446411

Train Step: 2000, Loss: 0.1092374324798584
toxic roc_auc 0.9782557487487793
severe_toxic roc_auc 0.985173225402832
obscene roc_auc 0.9869914650917053
threat roc_auc 0.940433144569397
insult roc_auc 0.9840552806854248
identity_hate roc_auc 0.951041042804718

Train Step: 3000, Loss: 0.08654267340898514
toxic roc_auc 0.9829816222190857
severe_toxic roc_auc 0.988050103187561
obscene roc_auc 0.9901209473609924
threat roc_auc 0.9722374677658081
insult roc_auc 0.9854304790496826
identity_hate roc_auc 0.9764814376831055

Train Step: 4000, Loss: 0.074321515



Epoch 1, Validation Loss: 0.038444582372903824, Time: 3846.4300816059113

toxic roc_auc 0.9880316257476807
severe_toxic roc_auc 0.9882777333259583
obscene roc_auc 0.991179347038269
threat roc_auc 0.977181077003479
insult roc_auc 0.9845747351646423
identity_hate roc_auc 0.9721691012382507




## 4. Run predictions on test-set & save submission

In [ ]:
test_input_ids = tokenize_sentences(df_test['comment_text'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

In [ ]:
TEST_BATCH_SIZE = 32
test_steps = len(df_test) // TEST_BATCH_SIZE

test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False, epochs=1)

df_submission = pd.read_csv(subm_path, index_col='id')

for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
    sample_ids = df_test.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['id']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_submission.loc[sample_ids, label_cols] = predictions

In [ ]:
df_submission.to_csv('submission.csv')

In [ ]:
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f submission.csv -m "transformer bert"

100% 21.3M/21.3M [00:00<00:00, 24.4MB/s]
Successfully submitted to Toxic Comment Classification Challenge

In [ ]:
!pip install lime

     |████████████████████████████████| 276kB 25.5MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283845 sha256=cd2089fc3b5300241e0f04806cbe49feae06e4faa6784fc12f829e6381be31d8
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [ ]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
import sklearn.feature_extraction
from __future__ import print_function

In [ ]:
input_ids = tokenize_sentences(pd.Series(text), tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
len(input_ids[0])

128

In [ ]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer
# define softmax function
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# define prediction function
def predict_probs(text):

  input_ids = tokenize_sentences(pd.Series(text), tokenizer, MAX_LEN)
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  attention_masks = create_attention_masks(input_ids)

  dataset = create_dataset((input_ids, attention_masks), batch_size=TEST_BATCH_SIZE, train=False, epochs=1)
  predictions = []
  for i, (token_ids, masks) in enumerate(tqdm(dataset, total=test_steps)):
    preds = model(token_ids, attention_mask=masks).numpy()[0][0]
    predictions += [np.array([preds, 1-preds])]
  
  x = np.array(list(predictions))
  print(type(x))
  print(type(x[0]))
  print(type(x[0][0]))
  print(x)
  return x

explainer = LimeTextExplainer(class_names=["toxic","clean"])
text = df_train["comment_text"].tolist()[0]
exp = explainer.explain_instance(text, predict_probs, num_features=6)

/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]



<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
[[1.23533246e-03 9.98764668e-01]
 [2.13630451e-03 9.97863695e-01]
 [1.11916987e-03 9.98880830e-01]
 [2.08519422e-03 9.97914806e-01]
 [2.02836376e-03 9.97971636e-01]
 [1.47172273e-03 9.98528277e-01]
 [1.56635942e-03 9.98433641e-01]
 [1.69259263e-03 9.98307407e-01]
 [3.00443452e-03 9.96995565e-01]
 [2.73647881e-03 9.97263521e-01]
 [1.84175186e-03 9.98158248e-01]
 [1.42766861e-03 9.98572331e-01]
 [3.18931486e-03 9.96810685e-01]
 [1.79944013e-03 9.98200560e-01]
 [1.08238333e-03 9.98917617e-01]
 [1.39825745e-03 9.98601743e-01]
 [3.07953283e-02 9.69204672e-01]
 [2.23768363e-03 9.97762316e-01]
 [4.12784982e-03 9.95872150e-01]
 [9.55586671e-04 9.99044413e-01]
 [2.53608078e-03 9.97463919e-01]
 [1.17002008e-03 9.98829980e-01]
 [1.31163851e-03 9.98688361e-01]
 [1.56908215e-03 9.98430918e-01]
 [1.13290781e-03 9.98867092e-01]
 [1.50214601e-03 9.98497854e-01]
 [1.63907893e-02 9.83609211e-01]
 [1.28739257e-03 9.98712607e-01]
 [1

ValueError: ignored